In [1]:
import json
import pandas as pd

import argparse
import io
import os
from datetime import date
# Imports the Google News api
from newsapi import NewsApiClient

# Imports the Google Cloud client library
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types


# Functions

In [2]:
def classify(text):
    language_client = language.LanguageServiceClient.from_service_account_json('htn2019-f059d4b46f7d.json')
    document = language.types.Document(content=text, type=language.enums.Document.Type.PLAIN_TEXT)
    response = language_client.classify_text(document)
    categories = response.categories

    result = {}

    for category in categories:
        categories = category.name.split("/")
        categories.pop(0)
        for i in range(len(categories)):
            result[categories[i]] = category.confidence

    return result

## Input Parameter

In [3]:
print("Enter the original currency: ")
# original = input()
print("Enter the new currency: ")
# return = input()

Enter the original currency: 
Enter the new currency: 


In [4]:
original = 'CAD'

## Article Retrieval 

In [5]:
newsapi = NewsApiClient(api_key='a93fd9a824b848bca3436e761c5a0d4d')

In [6]:
today = date.today().strftime("%Y/%m/%d").replace("/","-")

# grabbing articles
all_articles = newsapi.get_everything(q=original, from_param='2019-08-15', to=today, language='en', sort_by='relevancy')
articles = all_articles['articles']

In [7]:
formatted_articles = []

# creating new article object
for article in articles:
    info = str(article['title']) + " " + str(article['description']) + " " + str(article['content'])
    classifications = classify(info)
    print(classifications)
    print()
#     print(classifications)
    if classifications:
        individual_article = {}
        # just the date
        # print(article['publishedAt'][:10])
        individual_article['date'] = article['publishedAt']

        info = info.replace("'","") 
        individual_article['text'] = info

        formatted_articles.append(individual_article)


{'Computers & Electronics': 0.8700000047683716, 'Consumer Electronics': 0.8700000047683716, 'Internet & Telecom': 0.7200000286102295, 'Mobile & Wireless': 0.7200000286102295, 'Mobile Phones': 0.7200000286102295}

{'Finance': 0.8899999856948853, 'Investing': 0.8899999856948853, 'Currencies & Foreign Exchange': 0.8899999856948853}

{'Computers & Electronics': 0.8799999952316284, 'Consumer Electronics': 0.8799999952316284, 'Internet & Telecom': 0.8399999737739563, 'Mobile & Wireless': 0.8399999737739563, 'Mobile Phones': 0.8399999737739563}

{'Computers & Electronics': 0.9900000095367432, 'Consumer Electronics': 0.9900000095367432, 'Game Systems & Consoles': 0.9900000095367432, 'Games': 0.9900000095367432, 'Computer & Video Games': 0.9900000095367432}

{'Arts & Entertainment': 0.6600000262260437, 'Visual Art & Design': 0.6600000262260437}

{'Computers & Electronics': 0.6800000071525574}

{'Computers & Electronics': 0.550000011920929, 'Computer Hardware': 0.550000011920929, 'Computer Perip

## Sentiment Analysis (Tone Analyzer)

In [8]:
client = language.LanguageServiceClient.from_service_account_json('htn2019-f059d4b46f7d.json')

In [9]:
tone_analyzed_articles = []

# The text to analyze
for i in range(len(formatted_articles)):
    analyzed_article = {}
    article = formatted_articles[i]
    text = article['text']
    
    document = types.Document(content = text, type = enums.Document.Type.PLAIN_TEXT)
    
    # Detects the sentiment of the text
    sentiment = client.analyze_sentiment(document=document).document_sentiment
    
    analyzed_article['date'] = article['date']
    analyzed_article['score'] = sentiment.score
    analyzed_article['magnitude'] = sentiment.magnitude
    
    tone_analyzed_articles.append(analyzed_article)

print(json.dumps(tone_analyzed_articles, indent=4, sort_keys=True))

#     print('Text: {}'.format(text))
#     print('Sentiment: {}, {}'.format(, sentiment.magnitude))

[
    {
        "date": "2019-08-26T07:26:03Z",
        "magnitude": 1.0,
        "score": 0.10000000149011612
    },
    {
        "date": "2019-08-20T08:58:47Z",
        "magnitude": 2.5999999046325684,
        "score": -0.4000000059604645
    },
    {
        "date": "2019-09-04T12:56:11Z",
        "magnitude": 1.5,
        "score": 0.0
    },
    {
        "date": "2019-08-15T22:50:28Z",
        "magnitude": 1.100000023841858,
        "score": 0.20000000298023224
    },
    {
        "date": "2019-09-12T14:00:44Z",
        "magnitude": 2.0,
        "score": 0.20000000298023224
    },
    {
        "date": "2019-08-27T23:00:30Z",
        "magnitude": 1.5,
        "score": -0.20000000298023224
    },
    {
        "date": "2019-09-05T17:01:48Z",
        "magnitude": 2.5,
        "score": 0.20000000298023224
    },
    {
        "date": "2019-09-06T16:01:56Z",
        "magnitude": 2.5,
        "score": 0.6000000238418579
    },
    {
        "date": "2019-09-12T13:20:51Z",
        "ma